<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/CheckDiag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
hemo = pd.read_csv('/content/drive/MyDrive/ckdresearch2023/Hemodialysis.csv', encoding='latin1')

## **DiagSummary**

In [3]:
diagnosis = pd.read_csv('/content/drive/MyDrive/ckdresearch2023/Diagnosis.csv')

In [4]:
patient_uid_list = hemo['PatientUID'].unique().tolist()
diagnosis = diagnosis[~diagnosis['PatientUID'].isin(patient_uid_list)]

In [5]:
diagnosis0 = pd.DataFrame(diagnosis)

In [6]:
diag_code = diagnosis['ProblemCode'].value_counts().reset_index()
diag_code.columns = ['UniqueValue_CODE','Count']

diag_name = diagnosis['ProblemName'].value_counts().reset_index()
diag_name.columns = ['UniqueValue_NAME','Count']

# Specify the columns for unique value counting
columns_to_count = ['ProblemCode', 'ProblemName']

# Count unique values in the specified columns
diag_count = diagnosis[columns_to_count].value_counts().reset_index()
diag_count.columns = ['ProblemCode', 'ProblemName', 'Count']

diag_df = pd.DataFrame(diag_count)

In [7]:
DiagSummaryDF = pd.DataFrame(diag_df)

In [8]:
DiagSummaryDF

,ProblemCode,ProblemName,Count
0,370992007,Dyslipidemia,124826
1,I10,Essential (primary) hypertension,90673
2,44054006,Type 2 diabetes mellitus,68737
3,1201005,Benign essential hypertension,66796
4,59621000,Essential hypertension,51307
...,...,...,...
18914,31137003,"Early latent syphilis, positive serology, nega...",1
18915,311804006,Prolapsed lumbar intervertebral disc with scia...,1
18916,125587004,Superficial injury,1
18917,311892004,Collapse of vertebra due to osteoporosis NOS,1


## **Slope**
>> **ค่า x = ค่าจากวิสิท t ที่มาก่อน**

In [9]:
import pandas as pd
slope = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/SummaryFile/Slope.xlsx')

In [10]:
data = pd.DataFrame(slope)
data = data[['PatientUID','PatientVisitUID','Slopes']]

df = pd.DataFrame(data)

# Sorting the DataFrame by PatientUID and PatientVisitUID
df = df.sort_values(['PatientUID', 'PatientVisitUID'])

# Adding a NextSlope column
df['Slopes'] = df.groupby('PatientUID')['Slopes'].shift(-1)

data = df.dropna()
data

,PatientUID,PatientVisitUID,Slopes
0,2299,3386255,-7.158596
1,2299,4546471,5.550691
2,2299,5812902,-6.063710
3,2299,6947220,-12.812245
4,2299,8186767,28.287500
...,...,...,...
73378,3319961,9270330,-4.256753
73380,3319971,9270331,-0.692078
73382,3320048,9242721,7.217875
73384,3320050,9257281,-32.794557


# **DiagnosisList**

In [11]:
mapping = {
    'diag1': [1,3,4,10,81,395,510,1148,1668,2138,2318,2519,3269,3824,4084,4432,4534,5399,5853,6060,6913,7518,8336,8359,9084,10694,10869,12685,13526,13665,13781,15605,16508,16781,16900,16938,17271,17363],
    'diag2': [2,5,7,20,41,69,116,178,181,184,195,206,275,279,495,514,520,650,694,710,753,825,918,975,1061,1099,1142,1197,1242,1266,1389,1435,1466,1496,1509,1544,1585,1715,1723,1865,2095,2383,3186,3266,3284,3361,3413,3564,4004,4096,4474,4655,4719,5396,5541,5587,5770,5830,5856,5999,6059,6062,6248,6358,6374,6390,6391,6756,6833,6915,6916,7453,7568,7650,7986,8360,8402,8590,8633,8735,8983,9121,9303,9390,9739,9761,10082,10168,10558,10573,10577,10954,10977,10998,11772,12180,12651,12837,13020,13089,13417,13646,13916,14137,14907,14909,14911,14953,14958,15131,15282,16089,16238,16493,16762,17119,17157,18197,18809,18880,18881,18899,18902,18903,18904],
    'diag3': [0,514,17157,29,420,998,4353,6655,8189,8409,14889,14970,15011],
    'diag4': [288,375,428,1034,1562,7183,8062,54,109,416,1033,1413,1993,3929,4316,4987,5486,6754,7078,7667,11832],
    'diag5': [160,225,374,564,654,870,982,1114,2866,6424,9210,9879,13665,14495,18491],
    'diag6': [53,67,129,257,517,989,1066,1139,1385,1491,1641,1958,2010,2229,2286,2341,2435,3049,3184,3213,3285,3508,3812,4011,4170,4201,4378,4571,4790,4924,5212,5300,5914,5931,6236,6251,6262,6292,6614,6912,7849,7852,7998,8533,9300,9308,9383,10165,10187,10607,11070,11082,11084,11465,11671,11888,11911,12509,12542,12575,12580,12701,13381,13581,13998,14004,14173,14216,14218,14241,14254,14264,14270,14999],
    'diag7': [155,1438,1650,2062,2958,3084,3255,3618,3690,4229,4834,5348,5450,5590,6435,6790,7112,7530,9208,9400,10785,10892,12365,12370,12372,13470,13735,15508,18402,18403,18406,18407,18410,3398,3618,3851,3995,4200,4366,4376,6053,7153,8683,8755,9741,10013,11372,11401,12135,13748,14277,16026,16030,16086,16095,17151,17872,18349,18434,18754],
    'diag8': [32,196,528,1819,2108,2357,2705,7695,14483],
    'diag9': [42,44,534,1967,3456,11415,17132],
    'diag10': [211,887,1741,2000,3418,7013,9527,80,153,220,409,574,625,839,1401,1439,1771,1913,2335,3300,3423,3436,3595,3765,3877,3938,4174,4239,4939,5426,7927,8417,8559,8630,8883,8910,9559,9801,9885,11288,11986,12931,13200,15930,16146,17451,17929,18284],
    'diag11': [188,398,404,448,569,970,1011,1298,1503,2065,2354,2512,3232,3443,4812,4900,5367,5582,6001,7539,8010,8049,8447,8682,9176,10352,10601,10670,10745,13304,13404,13425,13442,13482,14717,16164,16165,16643,16741,17108,17217,17636,17842,18196],
    'diag12': [432,1024,1312,1349,1511,1938,2082,2176,2197,2353,2395,2520,2738,2744,3188,3882,4138,4637,5412,6425,7806,8405,8976,10386,12504,13130,13664,14794,17405],
    'diag13': [79,218,1189,1394,2117,2671,3895,3981,12549,13073,17037,376,378,1145,1194,1311,1321,1396,1640,1733,2462,2477,2756,3044,3478,5061,6011,7295,7848,7938,8664,10648,11261,11273,11750,11900,11946,12510,13529,14251,15280,15352,15462,15846,16372,17281,17560],
    'diag14': [240,355,400,1901-6-9,546,557,617,620,638,680,796,811,884,893,960,1102,1106,1167,1235,1276,1286,1292,1381,1443,1653,1821,1876,1954,2068,2221,2311,2401,2467,2533,2647,2819,3006,3118,3312,3355,3374,3491,3493,3617,3619,4173,4410,4411,4490,4502,4608,4970,5025,5100,5271,5465,5567,6009,6033,6161,6223,6282,6379,6418,6433,6597,6635,6636,6637,6678,7024,7028,7145,7304,7379,7419,7825,8009,8014,8049,8166,8283,8295,8640,8659,9796,9940,10200,10307,10308,10763,11085,11152,11192,11194,11462,11469,11753,11934,12084,12295,12541,12567,12572,12721,12723,13061,13204,13261,13321,13582,14186,14447,14451,15313,15567,15637,15897,15905,16412,16560,16849,16961,17100,17331,17334,17364,17410,17573,17832,18096,18363,18548,18606,18834],
    'diag15': [132,1625,8527,1925-11-27,17430,37,45,59,83,86,186,315,318,335,472,488,518,566,652,808,834,919,988,1055,1088,1202,1341,1369,1426,1548,1579,1621,1645,1776,1808,1826,1846,1873,2022,2141,2231,2258,2352,2460,2478,2567,2648,2725,2794,2881,2887,2908,3052,3066,3086,3119,3122,3233,3239,3278,3307,3313,3629,3698,3705,3751,3752,3846,3953,4045,4046,4070,4129,4193,4236,4240,4313,4338,4435,4520,4528,4564,4635,4671,4686,4723,4949,4965,5051,5169,5344,5467,5516,5750,5984,5991,6068,6163,6266,6407,6513,6515,6795,6836,6854,6864,6873,6922,6956,7038,7576,7588,7589,7664,7694,7760,7850,7886,8046,8094,8448,9130,9138,9148,9218,9355,9399,9467,9815,10079,10156,10393,10395,10413,10602,10747,10829,10835,10901,11419,11442,11453,11574,11585,11680,11815,11834,11928,11929,12192,12198,12212,12251,12526,12639,12761,12766,12767,13047,13344,13448,13519,13520,13604,13691,13810,13891,14707,16061,16756,17185,17231,17360,17361,17429,17430,17604,17855,17866,17971,17972,18004,18006,18152,18364,18405,18529,18615,18759],
    'diag16': [185,771,1378,1773,2500,2864,3713,3952,4034,4139,6117,7289,10328,11560,12297,14107,16121,16458,16581,205,435,464,541,842,974,1081,1350,1367,1422,1430,1969,2073,2182,2237,2238,2270,2316,2334,2390,2553,2670,2895,3002,3621,3643,3661,3680,3726,3932,4034,4087,4202,4210,4485,4487,4643,4661,4695,4905,4989,5034,5072,5140,5211,5223,5371,5451,5479,5600,5607,5644,6485,6704,6837,6870,7015,7321,7338,7854,7943,7945,8034,8064,8074,8103,8142,8434,8436,8530,8623,9008,9020,9279,9417,9465,9534,9598,10332,10401,11041,11043,11568,12128,12227,12646,12647,12691,13690,13707,13848,13853,13983,13984,14726,14727,14729,14748,15508,15550,15564,16459,16460,16592,16734,17017,17343,17504,17505,18337,18339,18341,18342,18344,18501,18906,36,306,797,6281,11284]
}

# Create the mapping of diagnosis DataFrames
diagnosis_mapping = {}
for diagnosis_name, indices in mapping.items():
    diagnosis_mapping[diagnosis_name] = DiagSummaryDF[DiagSummaryDF.index.isin(diagnosis.index[indices])]

# Create a new DataFrame to store the mapping values
mapping_data = []
for diagnosis_name, diag_df in diagnosis_mapping.items():
    result_item_names = diag_df['ProblemName'].tolist()
    mapping_data.extend([(diagnosis_name, name) for name in result_item_names])

mapping_df = pd.DataFrame(mapping_data, columns=['MappingValue', 'ProblemName'])

# Merge the mapping_df with the original diagnosis DataFrame based on 'ProblemName'
diagnosis = diagnosis.merge(mapping_df, on='ProblemName', how='left')


In [12]:
diagnosis = diagnosis[diagnosis['MappingValue'].notnull()]

In [13]:
diagnosis[['PatientUID','ProblemCode','ProblemName','ProblemDescription','MappingValue']]

,PatientUID,ProblemCode,ProblemName,ProblemDescription,MappingValue
4,22234,E789,"Disorder of lipoprotein metabolism\,unspecified",ความผิดปกติแบบอื่นของเมตะบอลิซึมของไลโปโปรตีน ...,diag2
6,22234,H811,Benign paroxysmal vertigo,การรู้สึกหมุนเป็นระยะแบบไม่รุนแรง,diag2
7,22234,H811,Benign paroxysmal vertigo,การรู้สึกหมุนเป็นระยะแบบไม่รุนแรง,diag3
8,17353,M329,"Systemic lupus erythematosus\, unspecified",ลูปัส อีริทิมาโตซัส ทั่วร่าง ไม่ระบุรายละเอียด,diag11
17,20936,E789,"Disorder of lipoprotein metabolism\,unspecified",ความผิดปกติแบบอื่นของเมตะบอลิซึมของไลโปโปรตีน ...,diag2
...,...,...,...,...,...
2085070,3065727,M10,Gout,NaN,diag1
2085080,2946464,44054006,Type 2 diabetes mellitus,NaN,diag1
2085083,2923976,90560007,Gout,NaN,diag1
2085087,3004348,312104005,Cholangiocarcinoma of biliary tract,NaN,diag9


# **DiagnosisTable**
> **จำนวนการเข้ารับการตรวจ**

In [15]:
import pandas as pd

# Group by PatientUID and PatientVisitUID and count each diagnosis
diagnosis_counts = diagnosis.groupby(['PatientUID', 'PatientVisitUID', 'MappingValue']).size().unstack(fill_value=0)

# Rename the columns to Diag1, Diag2, ..., Diag11
diagnosis_counts.columns = ['Diag{}'.format(col.split('diag')[1]) for col in diagnosis_counts.columns]

# Merge the diagnosis counts back into the data table
data = pd.merge(data, diagnosis_counts, how='left', on=['PatientUID', 'PatientVisitUID'])

# Fill NaN values with 0
data.fillna(0, inplace=True)


In [16]:
diagnosis[(diagnosis['PatientVisitUID'].isin([3386255, 4546471, 5812902, 6947220, 8186767])) &
                               (diagnosis['MappingValue'].isin(['diag1', 'diag2']))][['PatientUID','ProblemCode','ProblemName','MappingValue']]

,PatientUID,ProblemCode,ProblemName,MappingValue
23603,2299,1201005,Benign essential hypertension,diag2
23605,2299,239873007,Osteoarthritis of knee,diag2
34689,2299,1201005,Benign essential hypertension,diag2
34690,2299,239873007,Osteoarthritis of knee,diag2
47551,2299,239873007,Osteoarthritis of knee,diag2
59353,2299,239873007,Osteoarthritis of knee,diag2
72416,2299,239873007,Osteoarthritis of knee,diag2
72421,2299,59621000,Essential hypertension,diag1


In [20]:
data[data['PatientVisitUID'].isin([3386255, 4546471, 5812902, 6947220, 8186767])][['PatientUID','PatientVisitUID','Diag1','Diag2']]

,PatientUID,PatientVisitUID,Diag1,Diag2
0,2299,3386255,0.0,2.0
1,2299,4546471,0.0,2.0
2,2299,5812902,0.0,1.0
3,2299,6947220,0.0,1.0
4,2299,8186767,1.0,1.0


In [21]:
data.head()

,PatientUID,PatientVisitUID,Slopes,Diag1,Diag10,Diag11,Diag12,Diag13,Diag14,Diag15,Diag16,Diag2,Diag3,Diag4,Diag5,Diag6,Diag7,Diag8,Diag9
0,2299,3386255,-7.158596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2299,4546471,5.550691,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2299,5812902,-6.063710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2299,6947220,-12.812245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2299,8186767,28.287500,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### **RenameColumn**

In [22]:
DiagDF = pd.DataFrame(data)

In [23]:
column_rename_dict = {
    'Diag1': 'D1:Hypertension',
    'Diag2': 'D2:Diabetes mellitus',
    'Diag3': 'D3:Dyslipidemia',
    'Diag4': 'D4:SLE',
    'Diag5': 'D5:Systemic sclerosis',
    'Diag6': 'D6:Rheumatoid arthritis',
    'Diag7': 'D7:HIV infection',
    'Diag8': 'D8:HBV infection',
    'Diag9': 'D9:HCV infection',
    'Diag10': 'D10:Ischemic heart disease',
    'Diag11': 'D11:Heart failure',
    'Diag12': 'D12:Thalassemia',
    'Diag13': 'D13:Kidney stone',
    'Diag14': 'D14:Valvular heart disease',
    'Diag15': 'D15:Solid malignancy',
    'Diag16': 'D16:Hematologic malignancy'
}
# Use the rename method to rename the columns
DiagDF.rename(columns=column_rename_dict, inplace=True)
DiagDF

,PatientUID,PatientVisitUID,Slopes,D1:Hypertension,D10:Ischemic heart disease,D11:Heart failure,D12:Thalassemia,D13:Kidney stone,D14:Valvular heart disease,D15:Solid malignancy,D16:Hematologic malignancy,D2:Diabetes mellitus,D3:Dyslipidemia,D4:SLE,D5:Systemic sclerosis,D6:Rheumatoid arthritis,D7:HIV infection,D8:HBV infection,D9:HCV infection
0,2299,3386255,-7.158596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2299,4546471,5.550691,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2299,5812902,-6.063710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2299,6947220,-12.812245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2299,8186767,28.287500,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52615,3319961,9270330,-4.256753,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52616,3319971,9270331,-0.692078,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52617,3320048,9242721,7.217875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52618,3320050,9257281,-32.794557,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
DiagDF

,PatientUID,PatientVisitUID,Slopes,D1:Hypertension,D10:Ischemic heart disease,D11:Heart failure,D12:Thalassemia,D13:Kidney stone,D14:Valvular heart disease,D15:Solid malignancy,D16:Hematologic malignancy,D2:Diabetes mellitus,D3:Dyslipidemia,D4:SLE,D5:Systemic sclerosis,D6:Rheumatoid arthritis,D7:HIV infection,D8:HBV infection,D9:HCV infection
0,2299,3386255,-7.158596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2299,4546471,5.550691,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2299,5812902,-6.063710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2299,6947220,-12.812245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2299,8186767,28.287500,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52615,3319961,9270330,-4.256753,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52616,3319971,9270331,-0.692078,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52617,3320048,9242721,7.217875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52618,3320050,9257281,-32.794557,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
DiagDF = DiagDF[['PatientUID','PatientVisitUID','Slopes','D1:Hypertension','D2:Diabetes mellitus','D3:Dyslipidemia',
        'D4:SLE','D5:Systemic sclerosis','D6:Rheumatoid arthritis','D7:HIV infection','D8:HBV infection',
        'D9:HCV infection','D10:Ischemic heart disease','D11:Heart failure','D12:Thalassemia',
        'D13:Kidney stone','D14:Valvular heart disease','D15:Solid malignancy','D16:Hematologic malignancy']]

In [26]:
DiagDF.to_excel('DiagDF.xlsx')